In [44]:
import pandas as pd
import re
pd.__version__

u'0.24.2'

In [3]:
repL4L="/Users/gilles/ownCloud/Recherche/Boye/HDR/Data/L4L/"

# Nettoyage de la liste de formes verbales

## Ouverture

In [4]:
fListe="Lexique4linguists_MOT_VER.tsv"
dfListe=pd.read_csv(repL4L+fListe,sep="\t",encoding="utf8")
dfListe.head()
dfListe.LEMME=dfListe.LEMME.astype("unicode")
dfListe.INFOGRAM=dfListe.INFOGRAM.astype("str")
dfListe.FREQUENCE=dfListe.FREQUENCE.astype("int")

## Élimination des lexèmes fautifs

In [ ]:
lFautifs=dfListe.loc[~dfListe.LEMME.str.contains("re?$")]["ORTHO"].tolist()
dfListe.loc[dfListe.ORTHO.isin(lFautifs)]

In [ ]:
lInfinitifsFautifs=dfListe.loc[~dfListe.LEMME.str.contains("re?$")].index.tolist()

In [ ]:
dfListe.drop(lInfinitifsFautifs,inplace=True)

In [ ]:
lInfogramFautifs=dfListe[~dfListe.INFOGRAM.str.contains(":")&~dfListe.INFOGRAM.str.contains("^inf$")].index.tolist()

In [ ]:
dfListe.drop(lInfogramFautifs,inplace=True)

In [ ]:
lImperatif3Fautifs=dfListe[dfListe.INFOGRAM=='imp:pre:3s'].index.tolist()
dfListe.drop(lImperatif3Fautifs,inplace=True)

In [ ]:
cols=dfListe.INFOGRAM.unique().tolist()
cols

In [ ]:
mapGrace={}
mapBasilio={}
mapModes={"ind":"i","imp":"I","sub":"s","cnd":"c"}
mapTemps={"pre":"p","imp":"i","fut":"f","pas":"a"}
for col in cols:
    m=re.match(ur"(ind|imp|sub|cnd):(imp|pre|fut|pas):([123][sp])",col)
    if m:
        mapGrace[col]=mapTemps[m.group(2)]+mapModes[m.group(1)]+m.group(3).upper()
        mapBasilio[col]="Vm"+mapModes[m.group(1)].replace("I","m")+\
            mapTemps[m.group(2)].replace("a","s")+m.group(3).lower()+"-"
    elif col[:8]=="par:pas:":
        mapGrace[col]="pp"+col[8:].upper()
        mapBasilio[col]="Vmps-"+col[9].lower()+col[8].lower()
    elif col=="par:pre":
        mapGrace[col]="pP"
        mapBasilio[col]="Vmpp---"
    elif col=="inf":
        mapGrace[col]=col
        mapBasilio[col]="Vmn----"
    else:
        mapGrace[col]=col
        mapBasilio[col]=col
mapBasilio

In [ ]:
dfListe["grace"]=dfListe["INFOGRAM"].apply(lambda x: mapGrace[x])
dfListe["basilio"]=dfListe["INFOGRAM"].apply(lambda x: mapBasilio[x])

In [ ]:
dfListe.to_csv(repL4L+"L4L-Corrections.tsv",sep="\t",encoding="utf8")

In [11]:
dfListe=pd.read_csv(repL4L+"L4L-Corrections.tsv",sep="\t",encoding="utf8",index_col=0)

# Ajouter la phonologie de Flexique

In [12]:
# mapCols
import numpy as np

In [13]:
%store -r mapCols
fFlexique="vlexique.csv"
dfFlexique=pd.read_csv(repL4L+fFlexique,encoding="utf8")
dfFlexique.columns=[mapCols[c] if c in mapCols else c for c in dfFlexique.columns.tolist()]
dfFlexique.head(1)

,lexeme,variants,pi1S,pi2S,pi3S,pi1P,pi2P,pi3P,ii1S,ii2S,...,is3P,pI2S,pI1P,pI2P,inf,pP,ppMS,ppMP,ppFS,ppFP
0,abaisser,abaisser,abɛs,abɛs,abɛs,abEsɔ̃,abEse,abɛs,abEsE,abEsE,...,abEsas,abɛs,abEsɔ̃,abEse,abEse,abEsɑ̃,abEse,abEse,abEse,abEse


In [41]:
dfFlexique[dfFlexique.lexeme!=dfFlexique.variants]

,lexeme,variants,pi1S,pi2S,pi3S,pi1P,pi2P,pi3P,ii1S,ii2S,...,pI2S,pI1P,pI2P,inf,pP,ppMS,ppMP,ppFS,ppFP,phonoFlex
145,aicher,aicher:écher:escher,ɛʃ,ɛʃ,ɛʃ,Eʃɔ̃,Eʃe,ɛʃ,EʃE,EʃE,...,ɛʃ,Eʃɔ̃,Eʃe,Eʃe,Eʃɑ̃,Eʃe,Eʃe,Eʃe,Eʃe,fɔ̃
220,ancrer,ancrer:encrer,ɑ̃kʁ,ɑ̃kʁ,ɑ̃kʁ,ɑ̃kʁɔ̃,ɑ̃kʁe,ɑ̃kʁ,ɑ̃kʁE,ɑ̃kʁE,...,ɑ̃kʁ,ɑ̃kʁɔ̃,ɑ̃kʁe,ɑ̃kʁe,ɑ̃kʁɑ̃,ɑ̃kʁe,ɑ̃kʁe,ɑ̃kʁe,ɑ̃kʁe,ʃɑ̃ʒe
280,arder,arder:harder,aʁd,aʁd,aʁd,aʁdɔ̃,aʁde,aʁd,aʁdE,aʁdE,...,aʁd,aʁdɔ̃,aʁde,aʁde,aʁdɑ̃,aʁde,aʁde,aʁde,aʁde,diz
312,assener,assener:asséner,asɛn,asɛn,asɛn,asEnɔ̃,asEne,asɛn,asEnE,asEnE,...,asɛn,asEnɔ̃,asEne,asEne,asEnɑ̃,asEne,asEne,asEne,asEne,mɑ̃k
407,bagoter,bagoter:bagotter,bagɔt,bagɔt,bagɔt,bagOtɔ̃,bagOte,bagɔt,bagOtE,bagOtE,...,bagɔt,bagOtɔ̃,bagOte,bagOte,bagOtɑ̃,bagOte,bagOte,bagOte,bagOte,tʁavaj
411,bailler,bailler:bâiller:bayer;bâyer,baj,baj,baj,bajɔ̃,baje,baj,bajE,bajE,...,baj,bajɔ̃,baje,baje,bajɑ̃,baje,baje,baje,baje,asEje
468,becquer,becquer:béquer,bɛk,bɛk,bɛk,bEkɔ̃,bEke,bɛk,bEkE,bEkE,...,bɛk,bEkɔ̃,bEke,bEke,bEkɑ̃,bEke,bEke,bEke,bEke,tʁuv
469,becqueter,becqueter:becter,bEkt,bEkt,bEkt,bEktɔ̃,bEkte,bEkt,bEktE,bEktE,...,bEkt,bEktɔ̃,bEkte,bEkte,bEktɑ̃,bEkte,bEkte,bEkte,bEkte,aʁiv
537,bonir,bonir:bonnir,bOni,bOni,bOni,bOnisɔ̃,bOnise,bOnis,bOnisE,bOnisE,...,bOni,bOnisɔ̃,bOnise,bOniʁ,bOnisɑ̃,bOni,bOni,bOni,bOni,amyze
601,briefer,briefer:briffer,bʁif,bʁif,bʁif,bʁifɔ̃,bʁife,bʁif,bʁifE,bʁifE,...,bʁif,bʁifɔ̃,bʁife,bʁife,bʁifɑ̃,bʁife,bʁife,bʁife,bʁife,tɔ̃b


In [14]:
dfListe.head(1)

,ORTHO,LEMME,CATEGORIE,INFOGRAM,FREQUENCE,PHONOLOGIE,grace,basilio
0,est,être,VER,ind:pre:3s,6080693,e,pi3S,Vmip3s-


In [15]:
def phonoFlexique(row):
    lexeme=row["LEMME"]
    case=row.grace
    phono=dfFlexique[dfFlexique.lexeme.str.contains("^%s$"%lexeme)][case]
    if phono.empty:
        # print "no form",lexeme,case
        return np.NaN
    else:
        # print lexeme,case,phono.values[0]
        return phono.values[0]

In [16]:
dfListe["phonoFlex"]=dfListe.apply(phonoFlexique,axis=1)

In [17]:
dfListe[dfListe.LEMME.str.startswith("ex")&dfListe.phonoFlex.str.contains(u"^[^eɛE]")]

,ORTHO,LEMME,CATEGORIE,INFOGRAM,FREQUENCE,PHONOLOGIE,grace,basilio,phonoFlex


In [18]:
noLexeme=dfListe[dfListe.phonoFlex.isnull()&(dfListe.FREQUENCE>1)].LEMME.unique().tolist()
variants=dfFlexique.variants.unique().tolist()

In [19]:
lReparables={}
for lexeme in noLexeme:
    for variant in variants:
        lVariant=variant.split(":")
        if lexeme in lVariant:
            # print lexeme,variant,lVariant
            lReparables[lexeme]=variant
lReparables

{u'ass\xe9ner': u'assener:ass\xe9ner',
 u'becter': u'becqueter:becter',
 u'briffer': u'briefer:briffer',
 u'brinquebaler': u'bringuebaler:brinquebaler',
 u'butter': u'buter:butter',
 u'b\xe2iller': u'bailler:b\xe2iller:bayer;b\xe2yer',
 u'b\xe9quer': u'becquer:b\xe9quer',
 u'caller': u'caler:caller',
 u'canner': u'caner:canner',
 u'chopper': u'choper:chopper',
 u'clamser': u'clamecer:clamser',
 u'clapper': u'claper:clapper',
 u'coexister': u'co-exister:coexister',
 u'conter': u'compter:conter:',
 u'cosigner': u'co-signer:cosigner',
 u'crasher': u'cracher:crasher',
 u'dessaper': u'd\xe9saper:dessaper',
 u'desseller': u'desceller:desseller',
 u'dessouler': u'dessaouler:dessouler:desso\xfbler',
 u'desso\xfbler': u'dessaouler:dessouler:desso\xfbler',
 u'dribler': u'dribbler:dribler',
 u'd\xe9clencher': u'd\xe9clancher:d\xe9clencher',
 u'd\xe9ferrer': u'd\xe9f\xe9rer:d\xe9ferrer',
 u'd\xe9gotter': u'd\xe9goter:d\xe9gotter',
 u'd\xe9goutter': u'd\xe9go\xfbter:d\xe9goutter',
 u'd\xe9lasser': 

In [38]:
def phonoCompleter(row):
    result=row["phonoFlex"]
    lexeme=row["LEMME"]
    if row["phonoFlex"]!=row["phonoFlex"]:
        if lexeme in lReparables:
            variants=lReparables[lexeme]
            case=row.grace
            print lexeme,variants
            phono=dfFlexique[dfFlexique.variants.str.contains("^%s$"%variants)][case]
            if phono.empty:
                # print "no form",lexeme,case
                result=np.NaN
                dResiduels[lexeme]=phono.values[0]
            elif not lexeme.startswith("re") and phono.values[0].startswith(u"ʁə"):
                # print "pb de préf",lexeme,variants,case,phono.values[0]
                # print "troncation de la phono",phono.values[0][2:]
                result=phono.values[0][2:]
            else:
                # print "compléter",lexeme,case,phono.values[0]
                result=phono.values[0]
        else:
            result=np.NaN
            dResiduels[lexeme]=np.NaN
    # else:
    #     result=row["phonoFlex"]
    return result

In [34]:
dfListe[dfListe.phonoFlex.isnull()].apply(phonoCompleter,axis=1)

sentir ressentir:sentir
sembler ressembler:sembler
sentir ressentir:sentir
sentir ressentir:sentir
servir resservir:servir
sentir ressentir:sentir
servir resservir:servir
sauter ressauter:sauter
sentir ressentir:sentir
servir resservir:servir
sembler ressembler:sembler
sentir ressentir:sentir
sauter ressauter:sauter
sentir ressentir:sentir
sembler ressembler:sembler
sembler ressembler:sembler
sembler ressembler:sembler
servir resservir:servir
sentir ressentir:sentir
servir resservir:servir
servir resservir:servir
sauter ressauter:sauter
sembler ressembler:sembler
sentir ressentir:sentir
saisir ressaisir:saisir
saisir ressaisir:saisir
sentir ressentir:sentir
pêcher pécher:pêcher
sentir ressentir:sentir
servir resservir:servir
sembler ressembler:sembler
déclencher déclancher:déclencher
servir resservir:servir
déclencher déclancher:déclencher
sembler ressembler:sembler
sembler ressembler:sembler
servir resservir:servir
servir resservir:servir
semer ressemer:semer
sentir ressentir:sentir
s

50           NaN
69           NaN
70           NaN
164          sɑ̃
167        sɑ̃bl
187          NaN
299          NaN
319       sɑ̃tiʁ
369          sɑ̃
378       sɛʁviʁ
440          NaN
454          sɑ̃
460          sɛʁ
466          NaN
486         sOte
622          NaN
652          NaN
672          NaN
696        sɑ̃ti
707        sɛʁvi
719          NaN
723          NaN
744       sɑ̃blE
779          NaN
780        sɑ̃te
808          NaN
811          NaN
831          NaN
838          NaN
964          NaN
           ...  
108764       NaN
108767       NaN
108769       NaN
108785       NaN
108788       NaN
108802       NaN
108815       NaN
108818       NaN
108822       NaN
108831       NaN
108834       NaN
108835      pife
108842       NaN
108860       NaN
108863    ʒabOta
108864       NaN
108877       NaN
108878       NaN
108882     tɛ̃tE
108885       NaN
108899       NaN
108900       NaN
108907       NaN
108927       NaN
108930       NaN
108931       NaN
108932       NaN
108937       N

In [39]:
dResiduels={}
dfFlexique["phonoFlex"]=dfListe.apply(phonoCompleter,axis=1)
dResiduels

sentir ressentir:sentir
sembler ressembler:sembler
sentir ressentir:sentir
sentir ressentir:sentir
servir resservir:servir
sentir ressentir:sentir
servir resservir:servir
sauter ressauter:sauter
sentir ressentir:sentir
servir resservir:servir
sembler ressembler:sembler
sentir ressentir:sentir
sauter ressauter:sauter
sentir ressentir:sentir
sembler ressembler:sembler
sembler ressembler:sembler
sembler ressembler:sembler
servir resservir:servir
sentir ressentir:sentir
servir resservir:servir
servir resservir:servir
sauter ressauter:sauter
sembler ressembler:sembler
sentir ressentir:sentir
saisir ressaisir:saisir
saisir ressaisir:saisir
sentir ressentir:sentir
pêcher pécher:pêcher
sentir ressentir:sentir
servir resservir:servir
sembler ressembler:sembler
déclencher déclancher:déclencher
servir resservir:servir
déclencher déclancher:déclencher
sembler ressembler:sembler
sembler ressembler:sembler
servir resservir:servir
servir resservir:servir
semer ressemer:semer
sentir ressentir:sentir
s

{u'an\xe9mier': nan,
 u'contre-expertiser': nan,
 u'fen\xeatrer': nan,
 u'd\xe9sentrelacer': nan,
 u'volanter': nan,
 u'criser': nan,
 u'mouvementer': nan,
 u'fienter': nan,
 u'satiner': nan,
 u'd\xe9nuer': nan,
 u'capeler': nan,
 u'd\xe9sinhiber': nan,
 u'enrailler': nan,
 u'consoner': nan,
 u'instiguer': nan,
 u'duveter': nan,
 u'd\xe9ganter': nan,
 u'it\xe9rer': nan,
 u'hancher': nan,
 u'entr\xe9gorger': nan,
 u'indicer': nan,
 u'bossuer': nan,
 u'colliger': nan,
 u'boursicoter': nan,
 u'aposter': nan,
 u'basaner': nan,
 u'caleter': nan,
 u'coquer': nan,
 u'd\xe9cimaliser': nan,
 u'revouloir': nan,
 u'conventionner': nan,
 u'bolcheviser': nan,
 u'contre-braquer': nan,
 u'texturer': nan,
 u'enficher': nan,
 u'palisser': nan,
 u'oiseler': nan,
 u'refonctionner': nan,
 u'parisianiser': nan,
 u'refonder': nan,
 u'invaginer': nan,
 u'biquer': nan,
 u'boudiner': nan,
 u'sourdre': nan,
 u'endenter': nan,
 u'gaminer': nan,
 u'd\xe9fragmenter': nan,
 u'champagniser': nan,
 u'encloquer': nan,

In [40]:
dfListe[dfListe.LEMME.isin(lReparables)]

,ORTHO,LEMME,CATEGORIE,INFOGRAM,FREQUENCE,PHONOLOGIE,grace,basilio,phonoFlex
164,sens,sentir,VER,ind:pre:1s,52819,s@,pi1S,Vmip1s-,NaN
167,semble,sembler,VER,ind:pre:3s,52128,s@bl,pi3S,Vmip3s-,NaN
319,sentir,sentir,VER,inf,25218,s@tiR,inf,Vmn----,NaN
369,sent,sentir,VER,ind:pre:3s,21458,s@,pi3S,Vmip3s-,NaN
378,servir,servir,VER,inf,20865,sERviR,inf,Vmn----,NaN
454,sens,sentir,VER,ind:pre:2s,17184,s@,pi2S,Vmip2s-,NaN
460,sert,servir,VER,ind:pre:3s,16927,sER,pi3S,Vmip3s-,NaN
486,sauter,sauter,VER,inf,16012,sote,inf,Vmn----,NaN
696,senti,sentir,VER,par:pas:ms,10631,s@ti,ppMS,Vmps-sm,NaN
707,servi,servir,VER,par:pas:ms,10456,sERvi,ppMS,Vmps-sm,NaN


In [ ]:
dfListe[dfListe.phonoFlex.isnull()&(dfListe.FREQUENCE>1)]

# Corriger les transcriptions des verbes préfixés en EX

In [ ]:
fParadigme="L4L-Complet.csv"
dfParadigme=pd.read_csv(repL4L+fL4L,sep=";")
dfParadigme.drop(columns=["0"],inplace=True)

In [ ]:
dfParadigme.loc[~dfParadigme.lexeme.str.contains("re?$")]

In [ ]:
exPrefs=dfL4L.loc[(dfL4L.lexeme.str.startswith("ex"))&(dfL4L.inf.str.startswith("E")==False)].lexeme.tolist()

In [ ]:
for l in dfL4L.loc[(dfL4L.lexeme.str.startswith("ex"))&(dfL4L.inf.isnull())].index.tolist():
    print dfL4L.loc[l].dropna()